In [1]:
using Multilane
using POMDPs
using POMDPToolbox
using MCTS
using Distributions # for Normal
using DataArrays
using DataFrames
using DataFramesMeta

#using Interact
using Interact
using Reactive
#using Records
using AutoViz
using AutomotiveDrivingModels
using JLD
#using Cairo
#using Reel
#using GenerativeModels

In [2]:
#Set up problem configuration
nb_lanes = 4
pp = PhysicalParam(nb_lanes,lane_length=100.) #2.=>col_length=8
_discount = 0.9
nb_cars=10

#rmodel = NoCrashRewardModel()
rmodel  = NoCrashRewardModel(0.0,0,2.5,1,100.,4,2500,100)
#rmodel  = NoCrashRewardModel(0.0,0,2.5,1,0.0,4,2500,100)

#be = standard_uniform()
#dmodel = NoCrashIDMMOBILModel(nb_cars, pp, behaviors = be)
dmodel = NoCrashIDMMOBILModel(nb_cars, pp)

mdp = NoCrashMDP{NoCrashRewardModel}(dmodel, rmodel, _discount)

rng = MersenneTwister(5);

In [3]:
solver = DPWSolver(depth=25,
                    exploration_constant=5.0,
                    n_iterations=800);

In [4]:
planner = solve(solver, mdp);

In [5]:
record_times = []
start_time = []
start_dis = []

0-element Array{Any,1}

In [ ]:
for i = 1:500
  is = relaxed_initial_state(mdp);
  is.x = 0;
  is.t = 0;
  is.Control_Signal = true;
  steps = 200;
  sim = relaxed_initial_state_cz(mdp,is,planner,steps);

  if sim.state_hist[end].cars[1].y ==1.0
      push!(record_times,1)
  else
      push!(record_times,0)
  end
  state = deepcopy(sim.state_hist);
  st_step = 1
  while state[st_step].Control_Signal && st_step<199
    st_step += 1
  end
  if st_step < 199
      push!(start_time,st_step)
      push!(start_dis,2500-state[st_step].x)  
  else
      push!(start_time,0)
      push!(start_dis,0) 
  end   
  println(i)
end


In [ ]:
JLD.save("test_0504_1.jld","record_times",record_times,"start_time",start_time,"start_dis",start_dis)

In [ ]:

startd = []
for i = 1:400
    if start_time[i] >0 && start_dis[i]>0
        push!(startd,start_dis[i])
    end
end


In [ ]:
n = 0
for i = 1:400
    if record_times[i]==0 && start_time[i] == 0
        n +=1
    end
end
n

In [ ]:
using PyPlot
plot(startd)